# _Redes neuronales artificiales_

### Importaciones

In [7]:
import random
from math import e 

### Visión general de la implemetación
Se definen las clases **Neuron** y **ANN** las cuales representarán de forma general a las neuronas y a la red neuronal a entrenear. <br>
Dados estos modelos, un objeto de tipo ANN contendrá varios objetos de tipo Neuron.

### Clase Neuron
***
#### Métodos
- **init(n_inputs)**: Inicializa n_inputs pesos de la neurona con valores entre -0.5 y 0.5 de forma aleatoria.
- **compute_input(inputs)**: Calcula la combinación lineal entre los pesos y los valores de las inputs.
- **apply_sigmoid(computed_value)**: Aplica la función de sigmoide a computed_value.

#### Observaciones
- Se asume que el valor $w_{0}$ es el último en la lista de pesos para mayor facilidad a la hora de implemetación.


In [8]:
class Neuron:
    def __init__(self, n_inputs):
        self.weights = [round(random.uniform(-0.5,0.5),5) for i in range(n_inputs)]

#     def create(self, n_inputs):
#         self.weights = [round(random.uniform(-0.5,0.5),5) for i in range(n_inputs)]
        
    def compute_input(self, inputs):
        result = self.weights[-1]
        for i in range(len(self.weights)-1):
            result += self.weights[i] * inputs[i]
        return result
    
    def apply_sigmoid(self, computed_value):
        return 1 / (1 + e**(-computed_value))
    

### Clase ANN
***
#### Métodos
- **init(n_inputs, n_hidden, n_outputs)**: Inicializa las neuronas de la capa oculta y la capa de salida. Para la capa oculta se crean n_hidden neuronas cada una de las cuales tiene (n_inputs + 1) pesos, uno por cada columna en el conjunto de datos más el adicional por el peso $w_{0}$. Adicionalmente se crean n_outputs neuronas para la capa de salida, cada una con (n_hidden + 1) pesos. Esto significa que cada neurona en la capa de salida se conecta con cada neurona de la capa oculta.
- **forward_propagate(instance)**: Recibe en instance una instancia a clasificar, procesa la misma con cada neurona de la capa oculta, luego cada salida generada por las neuronas de la capa oculta son procesadas por cada una de las neuronas de la capa de salida, devolviendo finalmente el resultado de la clasificación.

#### Observaciones
- asdasd $w_{0}$ asdasd.

In [1]:
class ANN:
    def __init__(self, n_inputs, n_hidden, n_outputs):
        self.hidden_layer = [Neuron(n_inputs + 1) for i in range(n_hidden)]
        self.output_layer = [Neuron(n_hidden + 1) for i in range(n_outputs)]
        
    def forward_propagate(self, instance):
        
        inputs = instance
        hidden_layer_outputs = []
        for neuron in self.hidden_layer:
            computed_input = neuron.compute_input(inputs)
            neuron_output = neuron.apply_sigmoid(computed_input)
            hidden_layer_outputs.append(neuron_output)
            
        inputs = hidden_layer_outputs
        final_outputs = []
        for neuron in self.output_layer:
            computed_input = neuron.compute_input(inputs)
            neuron_output = neuron.apply_sigmoid(computed_input)
            final_outputs.append(neuron_output)
        
        return final_outputs